# 멀티모달 RAG를 활용한 복합 데이터 처리
## 멀티모달 RAG
- 모달리티는 이미지, 텍스트, 음성등 입력받거나 처리하는 데이터의 형태를 의미합니다.
- 멀티모달은 이러한 서로 다른 유형의 데이터를 동시에 사용하는 방식을 뜻합니다.
- LLM에 이미지를 비롯한 차트, 테이블등 풍부한 정보를 제공한다면 관련 문서를 검색할때 더욱 정확한 맥락을 파악할 수 있습니다.

### 멀티모달 RAG가 어려운 이유
- 일반적인 비즈니스 도메인에서 다루는 비정형 데이터는 고해상도 이미지가 담긴 슬라이드 또는 텍스트, 표, 차트, 다이어그램 등이 섞인 형태로 존재하며, PDF등 다양한 형식으로 분산되어 있는 경우가 많습니다.
- 각 형식마다 해결해야 할 문제가 다르며, 형식간의 정보를 효과적으로 관리하는 방법도 고민해야 합니다.
- 이미지를 예로 들면, 세부적인 디테일보다는 전반적인 이미제 중점을 두고 분석하게 됩니다.
- 보고서나 문서에서는 차트나 다이어그램처럼 정보가 밀집된 이미지가 포함될 수 있습니다.
- 각 수치가 의미하는 바를 해석하는 세부적인 분석뿐만 아니라 해당 분석이 이루어진 환경과 같은 추가적인 맥락도 고려해야 합니다.
- 따라서 파이프라인을 설계할 때 각 양식의 특성과 요구사항을 인식하고 이에 맞춰 적절히 처리해야 정보를 효율적으로 담을수 있습니다.

### 데이터 일관성
- 멀티모달RAG에서 또 다른 중요한 과제는 서로 다른 형식의 데이터를 일관성 있게 관리하는 방법입니다.
- 예를 들어, 문서 내에서 차트를 설명하는 텍스트와 차트 자체의 의미가 일치하는지 확인하는 것이 필수입니다.
- LLM에 전달되는 두가지 유형의 정보가 서로 상반된다면, 답변 정확도가 떨어질 수 있습니다.

## 멀티모달 RAG 구현 방법
- 멀티모달 RAG 파이프라인을 구축하는 데는 서로 다른 모달리티를 어떻게 LLM에게 인지시킬지에 따라 몇가지 주요 접근 방식이 있습니다.
- 모든 모달리티를 동일한 벡터 공간에 포함하기
- 모든 모달리티를 하나의 기본 모달리티로 묶기
- 서로 다른 모달리티를 별도의 저장소에서 다루기

### 모든 모달리티를 동일한 벡터 공간에 포함하기
- 이미지와 텍스트를 모두 효과적으로 처리하기 위해 CLIP(Contrastive Language-Image Pretraining)과 같은 멀티모달 모델을 사용하면, 두 모달리티를 동일한 벡터 공간에 인코딩할 수 있습니다.
- 이미지와 텍스트 모두 동일한 방식으로 임베딩할 수 있으며, 기존 텍스트 기반의 RAG 인프라를 그대로 유지하면서 임베딩 모델만 교체하여 다양한 모달리티를 수용할 수 있습니다.
- 임베딩 모델만 교체하면 되서 구현하기 쉽다는 장점이 있습니다.
- 하지만 이미지, 텍스트, 복잡한 표 등 다양한 콘텐츠를 정확하게 임베딩 할 수 있는 모델을 확보하는 것이 필수 입니다.

# CLIP
- Contrasitive Language-Image Pretraining 모델
- 오픈AI에서 개발한 인공지능 모델로 텍스트와 이미지를 동시에 처리할 수 있는 멀티모달 모델입니다.
- 주어진 이미지에 맞는 텍스트를 예측하거나 텍스트에 맞는 이미지를 찾아 낼 수 있습니다.

## CLIP의 구조
- 크게 이미지와 텍스트를 처리하는 두가지 인코더로 구성되며, 두 인코더에서 생성된 벡터를 비교하는 방식으로 동작합니다.
- 이미지 인코더: 이미지를 입력으로 받아 해당 이미지를 벡터로 변환합니다. 주로 ResNet이나 Vision Transformaer(ViT) 같은 비전 모델이 이미지 인코더로 사용됩니다.
- 텍스트 인코더: 텍스트를 입력으로 받아 해당 텍스트를 벡터로 변환합니다. 일반적으로 Transformer기반의 언어 모델이 텍스트 인코더로 사용됩니다.
- 대조 학습: 이미지와 텍스트를 각각 인코더를 통해 벡터로 변환한 후, 이 벡터들을 비교하여 학습합니다. 이미지-텍스트 쌍이 맞으면 유사하게, 그렇지 않으면 다르게 나타나도록 조정합니다.

## CLIP의 특징
- 일반화 능력: CLIP은 특정 작업에 대해 훈련되지 않았음에도 불구하고 다양한 이미지-텍스트 관련 작업에서 강력한 성능을 발휘합니다. 예를들어 텍스트 설명에 맞는 이미지를 찾는 작업이나, 이미지를 보고 적절한 설명을 생성하는 작업에서 뛰어난 성능을 보입니다.
- Zero-shot 학습: CLIP은 트정 작업에 대해 별도의 추가 훈련 없이도 기존에 학습된 지식을 사용해 새로운 작업을 수행할 수 있습니다. CLIP은 새로운 카테고리의 이미지를 보고도 적절하게 텍스트로 설명할 수 있습니다.
- 멀티모달 학습: CLIP은 이미지와 텍스트를 동시에 처리할 수 있는 모델이므로 다양한 응용 프로그램에서 활용할 수 있습니다. 예를 들어, 이미지 검색, 이미지 설명, 생성, 멀티모달 챗봇등에 적용할 수 있습니다.

## 모든 모달리티를 하나의 기본 모달리티로 표현하기
- 모든 데이터를 한 가지 유형의 데이터로 표현하는 것입니다.
- 이미지와 텍스트 데이터를 함께 처리할 때, 멀티모달 LLM을 사용하면 이미지를 설명하는 텍스트 요약을 생성할 수 있습니다.
- 주된 모달리티 하나를 기본으로 선택하고, 다른 모든 모달리티를 기본 모달리티로 표현하는 전략입니다.
- 장점은 이미지로부터 생성된 메타데이터가 명확하고 객관적인 질문에 효과적으로 대응할 수 있다는 점입니다.
- 이미지 임베딩을 위한 새로운 모델을 따로 조정하거나, 다양한 검색 결과의 순위를 조정하기 위한 추가 작업이 필요하지 않다는 이점이 있습니다.
- 전처리 과정에서 비용이 발생하고, 이미지의 내부적인 뉘앙스나 정보가 손실될 수 있다는 한계도 있습니다.

## 서로 다른 모달리티를 별도의 저장소에서 다루기
- 모든 모달리티를 동일한 벡터 공간에 포함하는 방법과, 모든 모달리티를 하나의 기본 모달리티로 표현하는 방법을 결합한 형태라고 할 수 있습니다.
- 이미지와 텍스트를 모두 다루는 멀티모달 임베딩 모델의 임베딩 결과는 벡터 저장소1에,
- 이미지를 텍스트로 묘사하는 텍스트 임베딩 모델의 임베딩 결과는 벡터 저장소2에 저장합니다.
- 이후 사용자가 이미지와 텍스트로 구성된 질문을 입력하면 두 가지 임베딩 벡터를 각각 얻어야 합니다.
- 첫번째 임베딩 벡터는 벡터 저장소1에서 유사한 문서를 검색하는데 활용합니다.
- 두번째 임베딩 벡터는 벡터 저장소2에서 유사한 문서를 검색하는데 활용합니다.
- 이렇게 얻어낸 검색 결과들을 모두 사용하거나 또는 리랭킹을 거쳐 상위 n개의 유사한 문서를 가려낸뒤 최종적으로 LLM모델에게 답변을 생성할 맥락으로 제공합니다.
- 앞선 방식보다는 전처리에 시간이 많이 걸리고 인프라 비용도 배로들지만 사용자에게 가장 질적으로 우수한 답변을 제공할 수 있습니다.

[ch03_MULTI_MODAL_RAG.ipynb](ch03_MULTI_MODAL_RAG.ipynb)